In [1]:
import pandas as pd
# this is to avoid getting warnings
pd.options.mode.chained_assignment = None

In [2]:
# load the file to a DataFrame
df = pd.read_excel('/home/macbuse/RECRUTEMENT/MCF_153_LAST.xls')

In [3]:
df.columns

Index(['Campagne', 'Session', 'Corps', 'Section', 'Autre section', 'Article',
       'N° emploi', 'Emplois liés', 'Profil', 'Job profile',
       'Research fields EURAXESS', 'Implant', 'Numéro SI local',
       'Numéro Gesup', 'Localisation', 'Vacance', 'Type de candidature',
       'Chgt de section', 'Civilité', 'Nom', 'Nom d'usage ou marital',
       'Prénom', 'Né(e) le', 'Unnamed: 23', 'Unnamed: 24', 'Rapporteur1',
       'Rapporteur2', 'Avis local', 'Lieu de naissance', 'N° candidat',
       'Référence qualif', 'Numen', 'Etat dossier', 'Détail suivi',
       'Situation professionnelle', 'Lieu d'exercice',
       'Ville ou pays d'exercice', 'Nationalité', 'Adresse1', 'Adresse2',
       'Adresse3', 'Code postal', 'Code étranger', 'Ville', 'Pays',
       'Téléphone', 'Autre tél', 'Fax', 'Email', 'Candidat local', 'Chercheur',
       'Titre thèse', 'Date soutenance', 'Lieu soutenance', 'Directeur Thèse',
       'Jury', 'N° de qualif', 'Activités enseignement', 'Theme', 'motif',
       

---

# Define string constants that we'll use later

- all strings are unicode in Python 3 this simplifies things a lot

https://docs.python.org/3/howto/unicode.html

In [109]:
title = "TEST"

_header_html = f'''<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0       
Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"> 
<html xmlns="http://www.w3.org/1999/xhtml" lang="fr" xml:lang="fr">                                                               
<head> <title>{title}</title>                                         
<meta http-equiv="content-type" content="text/html; charset=UTF-8" />                                                             
'''     


_header_tex = r'''\documentclass[a4paper, 12pt, titlepage]{article}
\usepackage[utf8]{inputenc}
\usepackage{amsmath,amssymb}
\usepackage[french]{babel}
\selectlanguage{french}
\usepackage{enumerate}
\usepackage{graphicx}


\newcommand{\R}{\mathbb{R}}
\newcommand{\C}{\mathbb{C}}
\def\no{\noindent}
\def\vs{\vskip 1cm}
\def\ben{\begin{enumerate}}
\def\een{\end{enumerate}}

\begin{document}
\pagestyle{empty}
'''




_thesis_fields =  ['Titre thèse', 'Date soutenance', 
                  'Lieu soutenance', 'Directeur Thèse', 
                   'Jury']                                 
                                                                           
  
_activity_fields = [ 'Situation professionnelle', 
                    'Activités enseignement','Activités administratives']   

_name_fields = ['Nom', 'Prénom', 'Né(e) le']

_web_fields = ['google','arxiv']


In [97]:
candidates[_name_fields]

,Nom,Prénom,Né(e) le
0,ADICEAM,LAKI MICHEL FAUSTIN,20/09/1988
1,ALDANA DOMINGUEZ,CLARA LUCIA,04/12/1976
2,ANNI,SAMUELE,20/06/1985
3,ARANCIBIA ROBERT,NICOLAS JOSE,08/07/1985
4,BALKANOVA,OLGA,27/06/1988
...,...,...,...
103,VENEZIANO,FRANCESCO,06/07/1984
104,VIU SOS,JUAN,04/08/1987
105,VOGLAIRE,YANNICK,06/09/1983
106,ZALAMANSKY,GABRIEL,07/07/1989


---

## Helper functions to generate requests

In [21]:
def fix_name(row):
    ss = "{Prénom} {Nom}".format(**row)
    ss = ss.replace('-',' ')
    return ss.replace(' ','+')

def arxiv(row):
    name = fix_name(row)
    url = f'https://arxiv.org/search/?query={name}'
    url += '&searchtype=all&abstracts=show&order=-announced_date_first&size=50'                  
    request = f'<a href = "{url}" target = "blank">'    
    request += '  {Prénom} {Nom} </a>'.format(**row)
    return request

def google(row):
    name = fix_name(row)
    #maybe should do a +math to disambiguate
    url = f'<a href = "https://www.google.fr/search?q={name}+math"'
    url += ' target = "blank">{Prénom} {Nom} </a>'.format(**row)
    return url

---

# Make a web page for all the candidates

In [113]:
# this is a trick to get only valid rows
candidates = df[~df.iloc[:,0].isnull()]

names = candidates[_name_fields]
theses  = candidates[_thesis_fields]
activity = candidates[_activity_fields]

# now generate some extra fields
dico_names = names.to_dict(orient='records')
candidates['web_name'] = [ fix_name(_) for _ in dico_names]
candidates['google'] = [ google(x) for x in  dico_names]
candidates['arxiv'] =  [ arxiv(x) for x in dico_names]

web = candidates[_web_fields]

def table2blocks(dff):
    '''write a dictionnary to rows in html'''
    dico = dff.to_dict(orient='records')
    
    blocks = []
    for row in dico:
        block = [f'<b>{key} : </b> {value} <br>\n' for key,value in row.items()]
        blocks.append('\n'.join(block) )
    return blocks


text_blocks = [ table2blocks(_) for _ in [names, web, theses, activity] ]

with open('web.htm','w') as fp:
    fp.write(_header_html)
    for candidate in zip(*text_blocks):
        fp.write('\n<br>\n'.join(candidate))
        fp.write('<br><hr><br>')
            

In [114]:
candidates['web_name']

0        LAKI+MICHEL+FAUSTIN+ADICEAM
1       CLARA+LUCIA+ALDANA+DOMINGUEZ
2                       SAMUELE+ANNI
3      NICOLAS+JOSE+ARANCIBIA+ROBERT
4                     OLGA+BALKANOVA
                   ...              
103              FRANCESCO+VENEZIANO
104                     JUAN+VIU+SOS
105                 YANNICK+VOGLAIRE
106               GABRIEL+ZALAMANSKY
107                    HUAFENG+ZHANG
Name: web_name, Length: 108, dtype: object

---

# Regular expressions 

convert the templates to Python3 using **re**

In [89]:
dd = df.to_dict(orient='records')
dd[0].keys()

dict_keys(['Campagne', 'Session', 'Corps', 'Section', 'Autre section', 'Article', 'N° emploi', 'Emplois liés', 'Profil', 'Job profile', 'Research fields EURAXESS', 'Implant', 'Numéro SI local', 'Numéro Gesup', 'Localisation', 'Vacance', 'Type de candidature', 'Chgt de section', 'Civilité', 'Nom', "Nom d'usage ou marital", 'Prénom', 'Né(e) le', 'Unnamed: 23', 'Unnamed: 24', 'Rapporteur1', 'Rapporteur2', 'Avis local', 'Lieu de naissance', 'N° candidat', 'Référence qualif', 'Numen', 'Etat dossier', 'Détail suivi', 'Situation professionnelle', "Lieu d'exercice", "Ville ou pays d'exercice", 'Nationalité', 'Adresse1', 'Adresse2', 'Adresse3', 'Code postal', 'Code étranger', 'Ville', 'Pays', 'Téléphone', 'Autre tél', 'Fax', 'Email', 'Candidat local', 'Chercheur', 'Titre thèse', 'Date soutenance', 'Lieu soutenance', 'Directeur Thèse', 'Jury', 'N° de qualif', 'Activités enseignement', 'Theme', 'motif', 'Activités recherche', 'Activités administratives', 'Autres diplômes', 'Travaux', 'Titres', 'N

# the original template 

I copy pasted it from thr bottom of the file

In [90]:
texTemplateMCF = r'''
\begin{center}
{\Large %(Corps)s %(N° emploi)s-%(Numéro Gesup)s: Rapport sur la candidature de }
\end{center}
\vs

\no
{\bf {\large Nom} :} %(Nom)s\\
{\bf Prénom :} %(Prénom)s\\
{\bf Date de naissance :} %(Né(e) le)s\\
{\bf Fonction actuelle :} %(Situation professionnelle)s,\\ %(Lieu d'exercice)s \\


\vs
\no
{\bf {\large Thèse} :} %(Titre thèse)s\\
{\bf Date :} %(Date soutenance)s\\
{\bf Etablissement :} %(Lieu soutenance)s\\
{\bf Directeur :} %(Directeur Thèse)s\\
{\bf Jury :} %(Jury)s\\
\\
\no
{\bf Mots clés :}\\
{\bf Avis sur l'audition du candidat :} d\'efavorable/possible/tr\`es favorable \\

\section{Publications}


\ben
\item \`a remplir
\een

{\textbf{  Informations fournies :} %(Travaux)s


\section{Activités enseignement}

\ben
\item \`a remplir
\een

{\textbf{  Informations fournies :}%(Activités enseignement)s


\section{Administration}

\ben
\item \`a remplir
\een

{\textbf{  Informations fournies :}%(Activités administratives)s



\section{Autres (encadrement, animation, contrats, prix)}
\ben
\item \`a remplir
\een


\section{Rapport scientifique}

\section*{Domaine de recherche}

%(Activités recherche)s

\section*{Avis sur le dossier}

\textit{Texte de synthèse du rapporteur}

\vs
\no
Date et signature :

\end{document}
'''

---

# regexp magic



In [120]:
import re

def callback(m):
    return '{{{}}}'.format(m.group(1))

ss = texTemplateMCF.replace('{','{{')
#standard string replacements
ss = ss.replace('}','}}')
#ss = ss.replace('\','\\')

#regexp to change the formatting to python3
ss = re.sub('%\((.*?)\)s',callback,ss)
print(ss[:500])



\begin{{center}}
{{\Large {Corps} {N° emploi}-{Numéro Gesup}: Rapport sur la candidature de }}
\end{{center}}
\vs

\no
{{\bf {{\large Nom}} :}} {Nom}\\
{{\bf Prénom :}} {Prénom}\\
{{\bf Date de naissance :}} {Né(e) le}\\
{{\bf Fonction actuelle :}} {Situation professionnelle},\\ {Lieu d'exercice} \\


\vs
\no
{{\bf {{\large Thèse}} :}} {Titre thèse}\\
{{\bf Date :}} {Date soutenance}\\
{{\bf Etablissement :}} {Lieu soutenance}\\
{{\bf Directeur :}} {Directeur Thèse}\\
{{\bf Jury :}} {Jury}\\
\\


---

# test with some random candidate


so 
- I dumped it to a file
- compiled this and it compiled  OK except for **babel**
- normally you should run this as a loop

```
for candidate in dd:
    pass
```

In [119]:
dd = candidates.to_dict(orient='records')
candidate = dd[4]
latex_report = ss.format(**candidate)

name = candidate['web_name'].replace('+','_')

with open( f"{name}.tex",'w') as fp:
    fp.write(_header_tex)
    fp.write(latex_report)

---

# here is how I read the templates

In [25]:
with open("/home/macbuse/RECRUTEMENT/gregTexData.py","r") as fp:
    ss = fp.read()

In [47]:
print(ss)

# -*- coding: utf-8 -*-

#IMPORTANT: if you add a template to this file please make it *raw utf8*
#by adding the letters ur before the string

#I had to split the TeX header from the rest and write it as ascii

PRESIDENT="Erwan Lanneau"
VOTANTS=14
NULS=0
OUI=14
NON=0
SCALE=.2



motifs = {'demission': '''le candidat a retir\'e sa candidature.''',
          'a' : ur'''Compte tenu du sujet de recherche du candidat
          une int\'egration au sein du laboratoire est difficilement envisageable.''',
'ab': ur'''Compte tenu du sujet de recherche du candidat,
          une int\'egration au sein du laboratoire est difficilement envisageable.''',
'b': ur'''Le  dossier du candidat  pr\'esente un projet de rechereche int\'eressant
par rapport aux th\'ematiques du laboratoire mais le comit\'e juge prudent d'attendre un an ou plus  pour suivre son \'evolution.
''',
'c': ur'''Le  niveau du concours \'etant tr\`es \'elev\'e cette ann\'ee
la qualit\'e du dossier  n'est pas suffisante pour demander l